In [36]:
import pandas as pd
import os
import glob
from rdflib import Graph
from urllib.request import urlopen
import subprocess
import time
from IPython.display import Markdown as md
from IPython.display import HTML
from pathlib import Path

# use ontobio!!
# pd.set_option('display.notebook_repr_html',False)
# https://github.com/jupyter/nbconvert/issues/946

In [37]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

# Mondo analysis

from ontobio.ontol_factory import OntologyFactory
mondo_file="/Users/matentzn/ws/mondo/src/ontology/mondo-qc.obo"
ofa = OntologyFactory()
ont = ofa.create(mondo_file)
ont.get_roots()[:3]

In [38]:
warnings = [Path(warn).stem for warn in glob.glob('../../sparql/*-warning.sparql')] 
NUMBER_ERRORS_SHOWN=15
PRIMARY_ONTOLOGY_FOR_QC="edit"

mondo="http://purl.obolibrary.org/obo/mondo.owl"
g = Graph()
g.parse("../mondo-qc.owl", format="xml")

In [39]:
def get_report(name):
    report_files= glob.glob('mondo-qc-*-%s.tsv' %(name))
    reports = []
    for f in report_files:
        try:
            df = pd.read_csv(f,sep="\t")
            df['ontology']=Path(f).stem.replace("-"+name,"").replace("mondo-qc-","")
            df.columns = [str(col).replace("?","") for col in df.columns]
            df.replace('[<]http[:][/][/]purl[.]obolibrary[.]org[/]obo[/]MONDO[_]','MONDO:', inplace=True,regex=True)
            df.replace('[>]$','', inplace=True, regex=True)
            reports.append(df)
        except pd.errors.EmptyDataError:
            print("WARNING! ", name, " is empty and has been skipped.")
    if reports:
        return pd.concat(reports)
    return pd.DataFrame(columns=['ontology'])

## Robot report analysis

In [40]:
dfx = get_report("robot-report-obo")

In [41]:
df = dfx.copy()
# Bug in ROBOT, wrong number of columns:
df.columns=['Rule Name', 'Subject', 'Property', 'Value','Trash','Ontology']
df['Level'] = df.index
df.reset_index(level=0, inplace=True)
del df['index']
del df['Trash']
df=df[df['Subject'].str.startswith('MONDO')]
props=df[['Level','Rule Name','Value','Ontology']].groupby(['Ontology','Level','Rule Name']).count().add_suffix('_count').reset_index()
props.sort_values(inplace=True, by=['Ontology','Value_count'], ascending=False)
props_pivot=props.pivot(index='Rule Name', columns='Ontology', values='Value_count')
cols = props_pivot.columns.tolist()
props_pivot[cols] = props_pivot[cols].fillna(0).astype(int)
props_pivot = pd.DataFrame(props_pivot.to_records())

## Overview of errors

This breaks down the errors in the generic ROBOT report. _Note that the whole report is filtered to only show results pertaining to MONDO classes themselves_. All tables are cut off at {{ NUMBER_ERRORS_SHOWN }} rows.

In [42]:
display(md("Table: Breakdown of the number of relationships in Mondo."))
display(HTML(props_pivot.to_html(index=False)))

Table: Breakdown of the number of relationships in Mondo.

Rule Name,2019,2020,current,edit,edit-owl
deprecated_class_reference,49,56,60,0,1
duplicate_definition,132,109,4,2,2
duplicate_exact_synonym,1662,1690,1688,1688,1688
duplicate_label,3,3,0,0,0
duplicate_scoped_synonym,1443,1579,1597,1595,1595
equivalent_pair,36995,37031,37604,0,37576
invalid_xref,4,0,0,0,0
lowercase_definition,391,387,300,300,300
missing_definition,0,0,0,0,0
missing_label,0,0,0,0,0


In [43]:
display(md("## Detailed breakdown of errors: Editors file"))
dfo = df[df['Ontology']==PRIMARY_ONTOLOGY_FOR_QC]
for rule in dfo['Rule Name'].unique():
    display(md("#### %s" % rule.replace("_"," ").capitalize()))
    dfd = dfo[dfo['Rule Name']==rule].head(NUMBER_ERRORS_SHOWN)
    display(HTML(dfd.to_html(index=False)))

# Revealed a more complex issue we need to sort first: https://github.com/monarch-initiative/mondo/issues/1979
# https://github.com/ontodev/robot/issues/733 # Ignore 0021108 0045024 until ticket resolved.

## Detailed breakdown of errors: Editors file

#### Duplicate definition

Rule Name,Subject,Property,Value,Ontology,Level
duplicate_definition,MONDO:0005318,IAO:0000115,"Oral aphthous ulcers typically present as painful, sharply circumscribed fibrin-covered mucosal defects with a hyperemic border.",edit,ERROR
duplicate_definition,MONDO:0001014,IAO:0000115,"A slowly progressing leukemia characterized by a clonal (malignant) proliferation of maturing and mature myeloid cells or mature lymphocytes. When the clonal cellular population is composed of myeloid cells, the process is called chronic myelogenous leukemia. When the clonal cellular population is composed of lymphocytes, it is classified as chronic lymphocytic leukemia, hairy cell leukemia, or T-cell large granular lymphocyte leukemia.",edit,ERROR


#### Multiple equivalent classes

Rule Name,Subject,Property,Value,Ontology,Level
multiple_equivalent_classes,MONDO:0021108,owl:equivalentClass,36e1f854-7e32-4f93-a0c6-789fa5fcd195genid330662,edit,ERROR
multiple_equivalent_classes,MONDO:0021108,owl:equivalentClass,36e1f854-7e32-4f93-a0c6-789fa5fcd195genid330666,edit,ERROR
multiple_equivalent_classes,MONDO:0045024,owl:equivalentClass,36e1f854-7e32-4f93-a0c6-789fa5fcd195genid357415,edit,ERROR
multiple_equivalent_classes,MONDO:0045024,owl:equivalentClass,36e1f854-7e32-4f93-a0c6-789fa5fcd195genid357419,edit,ERROR


...

Rule Name,Subject,Property,Value,Ontology,Level
duplicate_scoped_synonym,MONDO:0000155,oboInOwl:hasExactSynonym,rare inborn error of sequestering of triglyceride,edit,WARN
duplicate_scoped_synonym,MONDO:0000155,oboInOwl:hasRelatedSynonym,rare inborn error of sequestering of triglyceride,edit,WARN
duplicate_scoped_synonym,MONDO:0000242,oboInOwl:hasExactSynonym,dermatophytosis of beard,edit,WARN
duplicate_scoped_synonym,MONDO:0000242,oboInOwl:hasRelatedSynonym,dermatophytosis of beard,edit,WARN
duplicate_scoped_synonym,MONDO:0000270,oboInOwl:hasExactSynonym,disorder of lower respiratory tract,edit,WARN
duplicate_scoped_synonym,MONDO:0000270,oboInOwl:hasRelatedSynonym,disorder of lower respiratory tract,edit,WARN
duplicate_scoped_synonym,MONDO:0000330,oboInOwl:hasExactSynonym,endemic typhus Fever,edit,WARN
duplicate_scoped_synonym,MONDO:0000330,oboInOwl:hasRelatedSynonym,endemic typhus Fever,edit,WARN
duplicate_scoped_synonym,MONDO:0000351,oboInOwl:hasExactSynonym,rare inborn error of methionine catabolic process,edit,WARN
duplicate_scoped_synonym,MONDO:0000351,oboInOwl:hasRelatedSynonym,rare inborn error of methionine catabolic process,edit,WARN


#### Lowercase definition

Rule Name,Subject,Property,Value,Ontology,Level
lowercase_definition,MONDO:0000712,IAO:0000115,frontotemporal dementia plus amyotrophic lateral sclerosis.,edit,INFO
lowercase_definition,MONDO:0001554,IAO:0000115,secondary glaucoma caused by either excessive size or spheric shape of the lens.,edit,INFO
lowercase_definition,MONDO:0001709,IAO:0000115,sarcoidosis with a complication of hypercalcemia.,edit,INFO
lowercase_definition,MONDO:0001875,IAO:0000115,inflammation of the lateral epicondyle.,edit,INFO
lowercase_definition,MONDO:0002121,IAO:0000115,neuritis of a single nerve.,edit,INFO
lowercase_definition,MONDO:0004135,IAO:0000115,thyroiditis associated with painless enlargement of the thyroid gland. It occurs more frequently in females and is characterized by alterations between hyperthyroidism and hypothyroidism and the eventual return to normal thyroid gland function.,edit,INFO
lowercase_definition,MONDO:0005196,IAO:0000115,presence of structurally anomalous spermatozoa in the semen; malformations include the physical bending of the sperm to produce kinks or bends,edit,INFO
lowercase_definition,MONDO:0005448,IAO:0000115,liver injury resulting from hepatitis C infection,edit,INFO
lowercase_definition,MONDO:0005483,IAO:0000115,hair loss as a result of chemotherapy treatment,edit,INFO
lowercase_definition,MONDO:0005490,IAO:0000115,stroke caused by the blockage of blood flow in one of the large arteries feeding the brain,edit,INFO


#### Missing superclass

Rule Name,Subject,Property,Value,Ontology,Level
missing_superclass,MONDO:0000065,rdfs:subClassOf,NaN,edit,INFO
missing_superclass,MONDO:0000070,rdfs:subClassOf,NaN,edit,INFO
missing_superclass,MONDO:0000162,rdfs:subClassOf,NaN,edit,INFO
missing_superclass,MONDO:0000224,rdfs:subClassOf,NaN,edit,INFO
missing_superclass,MONDO:0000252,rdfs:subClassOf,NaN,edit,INFO
missing_superclass,MONDO:0000261,rdfs:subClassOf,NaN,edit,INFO
missing_superclass,MONDO:0000263,rdfs:subClassOf,NaN,edit,INFO
missing_superclass,MONDO:0000266,rdfs:subClassOf,NaN,edit,INFO
missing_superclass,MONDO:0000405,rdfs:subClassOf,NaN,edit,INFO
missing_superclass,MONDO:0000421,rdfs:subClassOf,NaN,edit,INFO


## Other Quality Control checks

In [44]:
summary_data = []
all_tables = {}

for r in warnings:
    df_r = get_report(r)
    cols = df_r.columns.tolist()
    cols_term = df_r.columns.tolist()
    if 'term' in cols:
        cols.remove('term')
    if 'term_label' in cols:
        cols.remove('term_label')
        cols_term.remove('term_label')
    for o in df_r['ontology'].unique():
        df_ro=df_r[df_r['ontology']==o]
        summary_data.append([r,o,len(df_ro)])
    all_tables[r] = df_r

df_summary_data = pd.DataFrame(summary_data,columns=['check','ontology','errors'])
props_pivot=df_summary_data.pivot(index='check', columns='ontology', values='errors')
props_pivot = pd.DataFrame(props_pivot.to_records())
display(HTML(props_pivot.to_html(index=False)))

WARNING!  excluded-subsumption-is-inferred-warning  is empty and has been skipped.
WARNING!  excluded-subsumption-is-inferred-warning  is empty and has been skipped.
WARNING!  excluded-subsumption-is-inferred-warning  is empty and has been skipped.
WARNING!  excluded-subsumption-is-inferred-warning  is empty and has been skipped.
WARNING!  excluded-subsumption-is-inferred-warning  is empty and has been skipped.


check,2019,2020,current,edit,edit-owl
no-subclass-between-genetic-disease-warning,214,71,68,13,68
related-exact-synonym-warning,200,200,200,200,200
single-child-warning,1637,1580,1559,1384,1557


In [45]:
for warning in all_tables:
    df_r = all_tables[warning] 
    df_ro=df_r[df_r['ontology']==PRIMARY_ONTOLOGY_FOR_QC]
    if (len(df_ro)>0):
        display(md("### Check: %s" % r))
        display(md("#### %s (Errors: %d)" % (o,len(df_ro))))
        df_rd = df_ro.head(NUMBER_ERRORS_SHOWN).copy()
        display(HTML(df_rd.to_html(index=False)))

### Check: related-exact-synonym-warning

#### edit (Errors: 13)

term,term_label,d2,d2_label,ontology
MONDO:0009711,congenital fiber-type disproportion myopathy,MONDO:0100084,actinopathy,edit
MONDO:0008477,"spondylometaphyseal dysplasia, Kozlowski type",MONDO:0009943,Pyle disease,edit
MONDO:0009711,congenital fiber-type disproportion myopathy,MONDO:0100084,actinopathy,edit
MONDO:0010486,"palmoplantar keratoderma, mutilating, with periorificial keratotic plaques, X-linked",MONDO:0019014,mutilating palmoplantar keratoderma with periorificial keratotic plaques,edit
MONDO:0009711,congenital fiber-type disproportion myopathy,MONDO:0100084,actinopathy,edit
MONDO:0044304,hyperphenylalaninemia due to DNAJC12 deficiency,MONDO:0009861,phenylketonuria,edit
MONDO:0011211,axial spondylometaphyseal dysplasia,MONDO:0009943,Pyle disease,edit
MONDO:0012160,spondylometaphyseal dysplasia-cone-rod dystrophy syndrome,MONDO:0009943,Pyle disease,edit
MONDO:0009597,"metaphyseal chondrodysplasia, Spahr type",MONDO:0009943,Pyle disease,edit
MONDO:0009652,mucolipidosis type III gamma,MONDO:0018931,mucolipidosis type III,edit


### Check: related-exact-synonym-warning

#### edit (Errors: 1384)

term,total,ontology
MONDO:0021202,1,edit
MONDO:0007481,1,edit
MONDO:0002326,1,edit
MONDO:0021468,1,edit
MONDO:0003960,1,edit
MONDO:0011140,1,edit
MONDO:0021203,1,edit
MONDO:0021469,1,edit
MONDO:0002894,1,edit
MONDO:0006282,1,edit


### Check: related-exact-synonym-warning

#### edit (Errors: 200)

term,term_label,related,ontology
MONDO:0014756,"tremor, hereditary essential, 5",ETM5,edit
MONDO:0006999,tooth disease,disorder of calcareous tooth,edit
MONDO:0009424,Bartter disease type 2,hyperprostaglandin E syndrome 2,edit
MONDO:0017686,inborn aminoacylase deficiency,rare inborn error of aminoacylase activity,edit
MONDO:0020680,acute bronchiolitis,acute bronchiolitis,edit
MONDO:0000651,thoracic disease,disorder of thoracic segment of trunk,edit
MONDO:0003832,complement deficiency,disorder of complement activation,edit
MONDO:0002031,cecal disease,disorder of caecum,edit
MONDO:0011018,brachyolmia-amelogenesis imperfecta syndrome,DASS,edit
MONDO:0011012,African iron overload,iron overload in Africa,edit


## Mondo Statistics

In [46]:
df = get_report("term-stats")
df.drop_duplicates(inplace=True)

## Relationships

In [47]:
props=df[['term','property','ontology']].groupby(['ontology','property']).count().add_suffix('_count').reset_index()
props.sort_values(inplace=True, by=['ontology','term_count'], ascending=False)
props_pivot=props.pivot(index='property', columns='ontology', values='term_count')
props_pivot = pd.DataFrame(props_pivot.to_records())
cols = props_pivot.columns.tolist()
cols.remove('property')
props_pivot[cols] = props_pivot[cols].fillna(0).astype(int)
props_pivot.sort_values(by=[PRIMARY_ONTOLOGY_FOR_QC],ascending=False,inplace=True)
display(md("### Table: Breakdown of the number of relationships in Mondo."))
display(HTML(props_pivot.to_html(index=False)))

### Table: Breakdown of the number of relationships in Mondo.

property,2019,2020,current,edit,edit-owl
<http://www.geneontology.org/formats/oboInOwl#hasDbXref,99234,99326,99965,128446,99967
<http://www.geneontology.org/formats/oboInOwl#hasExactSynonym,63390,63636,64352,64376,64376
<http://www.w3.org/2000/01/rdf-schema#subClassOf,46835,46891,47538,46998,47544
<http://www.geneontology.org/formats/oboInOwl#hasRelatedSynonym,37814,37757,37872,37876,37876
<http://www.w3.org/1999/02/22-rdf-syntax-ns#type,23555,23623,24229,24232,24232
<http://www.geneontology.org/formats/oboInOwl#id,23443,23510,24112,24115,24115
<http://www.w3.org/2000/01/rdf-schema#label,23442,23510,24112,24115,24115
<http://purl.obolibrary.org/obo/IAO_0000115,15425,15452,15492,15493,15493
<http://www.geneontology.org/formats/oboInOwl#inSubset,13335,13383,13454,13453,13453
<http://www.w3.org/2002/07/owl#equivalentClass,44290,44367,45007,7403,44979


In [48]:
current_to_release_diff=props_pivot[['property', 'edit-owl', 'current']]
current_to_release_diff=current_to_release_diff[current_to_release_diff['current']!=current_to_release_diff['edit-owl']]
current_to_release_diff.dropna(inplace=True)
current_to_release_diff['difference']=current_to_release_diff['edit-owl']-current_to_release_diff['current']
current_to_release_diff.sort_values(by=['difference'],ascending=False,inplace=True)
current_to_release_diff[['edit-owl', 'current','difference']] = current_to_release_diff[['edit-owl', 'current','difference']].astype(int)
display(md("### Table: Current release vs Edit release: Differences in the number of specific relations"))
display(HTML(current_to_release_diff.to_html(index=False)))

### Table: Current release vs Edit release: Differences in the number of specific relations

property,edit-owl,current,difference
<http://www.geneontology.org/formats/oboInOwl#hasExactSynonym,64376,64352,24
<http://www.w3.org/2000/01/rdf-schema#subClassOf,47544,47538,6
<http://www.w3.org/2002/07/owl#deprecated,1945,1941,4
<http://www.geneontology.org/formats/oboInOwl#hasRelatedSynonym,37876,37872,4
<http://www.w3.org/2000/01/rdf-schema#seeAlso,3140,3136,4
<http://www.w3.org/1999/02/22-rdf-syntax-ns#type,24232,24229,3
<http://www.geneontology.org/formats/oboInOwl#id,24115,24112,3
<http://www.w3.org/2000/01/rdf-schema#label,24115,24112,3
<http://www.geneontology.org/formats/oboInOwl#created_by,66,63,3
<http://www.geneontology.org/formats/oboInOwl#hasBroadSynonym,486,483,3
